In [1]:
cd ..

/Users/chengjun/OneDrive - 南京大学/10papers/causal_inference_references/RebelCa
> nal157781-V1/Program


In [2]:
cd ..

/Users/chengjun/OneDrive - 南京大学/10papers/causal_inference_references/RebelCa
> nal157781-V1


In [3]:
adopath ++ "/Users/chengjun/Library/Application Support/Stata/ado/plus/"


  [1]              "/Users/chengjun/Library/Application Support/Stata/ado/plus/"
  [2]  (BASE)      "/Applications/Stata/ado/base/"
  [3]  (SITE)      "/Applications/Stata/ado/site/"
  [4]              "."
  [5]  (PERSONAL)  "/Users/chengjun/Documents/Stata/ado/personal/"
  [6]  (PLUS)      "/Users/chengjun/Library/Application Support/Stata/ado/plus/"
  [7]  (OLDPLACE)  "~/ado/"
  [8]              "/opt/anaconda3/lib/python3.8/site-packages/stata_kernel/ado"


In [4]:
**************************************************************************
*** Set up
**************************************************************************
use "Data/Final/rebellion.dta", clear
run "Program/Analysis/generalsetup.do"


In [5]:
**************************************************************************
*** Table2. Canal closure and rebellions: pre-treatment trends
**************************************************************************
keep if year>=1776 & year<=1825
gen pretrend=alongcanal*year
label variable pretrend "$ Along Canal \times Year $ "
global X pretrend

tab OBJECTID
scalar groups=r(r)
su $Y
scalar ymean=r(mean)



(121,900 observations deleted)





      ID of |
   Counties |      Freq.     Percent        Cum.
------------+-----------------------------------
          1 |         50        0.17        0.17
        322 |         50        0.17        0.35
        323 |         50        0.17        0.52
        324 |         50        0.17        0.70
        325 |         50        0.17        0.87
        326 |         50        0.17        1.04
        327 |         50        0.17        1.22
        328 |         50        0.17        1.39
        329 |         50        0.17        1.57
        330 |         50        0.17        1.74
        331 |         50        0.17        1.91
        332 |         50        0.17        2.09
        333 |         50        0.17        2.26
        334 |         50        0.17        2.43
        335 |         50        0.17        2.61
        336 |         50        0.17        2.78
        337 |         50        0.17        2.96
        338 |     

In [6]:
*** Main estimates
reghdfe $Y $X, absorb(i.OBJECTID i.year) cluster(OBJECTID)
eststo est1
qui tab OBJECTID if e(sample)
scalar groups=r(r)
qui su $Y if e(sample)
scalar ymean=r(mean)
estadd scalar depavg=ymean:est1
estadd scalar N_g=groups:est1

reghdfe $Y $X, absorb(i.OBJECTID i.year c.ashprerebels#i.year) cluster(OBJECTID)
eststo est2
qui tab OBJECTID if e(sample)
scalar groups=r(r)
qui su $Y if e(sample)
scalar ymean=r(mean)
estadd scalar depavg=ymean:est2
estadd scalar N_g=groups:est2

reghdfe $Y $X, absorb(i.OBJECTID i.year c.ashprerebels#i.year i.provid#i.year) cluster(OBJECTID)
eststo est3
qui tab OBJECTID if e(sample)
scalar groups=r(r)
qui su $Y if e(sample)
scalar ymean=r(mean)
estadd scalar depavg=ymean:est3
estadd scalar N_g=groups:est3

reghdfe $Y $X, absorb(i.OBJECTID i.year c.ashprerebels#i.year i.provid#i.year i.prefid#c.year) cluster(OBJECTID)
eststo est4
qui tab OBJECTID if e(sample)
scalar groups=r(r)
qui su $Y if e(sample)
scalar ymean=r(mean)
estadd scalar depavg=ymean:est4
estadd scalar N_g=groups:est4



(MWFE estimator converged in 2 iterations)

HDFE Linear regression                            Number of obs   =     26,800
Absorbing 2 HDFE groups                           F(   1,    535) =       0.27
Statistics robust to heteroskedasticity           Prob > F        =     0.6068
                                                  R-squared       =     0.0349
                                                  Adj R-squared   =     0.0134
                                                  Within R-sq.    =     0.0000
Number of clusters (OBJECTID) =        536        Root MSE        =     0.2962

                             (Std. err. adjusted for 536 clusters in OBJECTID)
------------------------------------------------------------------------------
             |               Robust
ash~ypop1600 | Coefficient  std. err.      t    P>|t|     [95% conf. interval]
-------------+----------------------------------------------------------------
    pretrend |   .0003023   .0005871     0.51   0

In [7]:
*** Get indicators of FEs
estfe est1 est2 est3 est4

In [8]:
*** Get Conley standard errors
preserve
hdfe $Y $X, clear absorb(i.OBJECTID i.year) tol(0.001) keepvars(OBJECTID year Y_COORD X_COORD)
ols_spatial_HAC $Y $X, lat(Y_COORD) lon(X_COORD) time(year) panel(OBJECTID) distcutoff(500) lagcutoff(50) disp star
matrix V_spat=vecdiag(e(V)) 
matmap V_spat SE_spat, m(sqrt(@)) 
estadd matrix sesp=SE_spat: est1 
restore

preserve
hdfe $Y $X, clear absorb(i.OBJECTID i.year c.ashprerebels#i.year) tol(0.001) keepvars(OBJECTID year Y_COORD X_COORD)
ols_spatial_HAC $Y $X, lat(Y_COORD) lon(X_COORD) time(year) panel(OBJECTID) distcutoff(500) lagcutoff(50) disp star
matrix V_spat=vecdiag(e(V)) 
matmap V_spat SE_spat, m(sqrt(@)) 
estadd matrix sesp=SE_spat: est2
restore

preserve
hdfe $Y $X, clear absorb(i.OBJECTID i.year c.ashprerebels#i.year i.provid#i.year) tol(0.001) keepvars(OBJECTID year Y_COORD X_COORD)
ols_spatial_HAC $Y $X, lat(Y_COORD) lon(X_COORD) time(year) panel(OBJECTID) distcutoff(500) lagcutoff(50) disp star
matrix V_spat=vecdiag(e(V)) 
matmap V_spat SE_spat, m(sqrt(@)) 
estadd matrix sesp=SE_spat: est3
restore

preserve
hdfe $Y $X, clear absorb(i.OBJECTID i.year c.ashprerebels#i.year i.provid#i.year i.prefid#c.year) tol(0.001) keepvars(OBJECTID year Y_COORD X_COORD)
ols_spatial_HAC $Y $X, lat(Y_COORD) lon(X_COORD) time(year) panel(OBJECTID) distcutoff(500) lagcutoff(50) disp star
matrix V_spat=vecdiag(e(V)) 
matmap V_spat SE_spat, m(sqrt(@)) 
estadd matrix sesp=SE_spat: est4
restore



(cachevars: year OBJECTID)
(converged in 3 iterations)

 
OLS REGRESSION
 
SE CORRECTED FOR CROSS-SECTIONAL SPATIAL DEPENDANCE
             AND PANEL-SPECIFIC SERIAL CORRELATION
 
DEPENDANT VARIABLE: ashonset_cntypop1600
INDEPENDANT VARIABLES:  pretrend
 
SPATIAL CORRELATION KERNAL CUTOFF: 500 KM
SERIAL CORRELATION KERNAL CUTOFF: 50 PERIODS
------------------------------------------------------------------------------
ashonse~1600 | Coefficient  Std. err.      t    P>|t|     [95% conf. interval]
-------------+----------------------------------------------------------------
    pretrend |   .0003023   .0006743     0.45   0.654    -.0010194     .001624
------------------------------------------------------------------------------
 
STANDARD ERRORS UNDER OLS, WITH SPATIAL CORRECTION AND WITH SPATIAL AND SERIAL C
> ORRECTION:

-----------------------------------------------
    Variable |   OLS      spatial    spatHAC   
-------------+---------------------------------
    pretrend |    0

In [9]:
**************************************************************************
*** Set up table elements for Latex
**************************************************************************
*** Title
global caption "Canal closure and rebellions: pre-treatment trends"

*** Table head
#delimit ;
global head
    \begin{table}[htb]\centering
    \caption{$caption }  \begin{threeparttable}
	\begin{tabular}{l*{4}{c}}
	\toprule\toprule
	&\multicolumn{4}{c}{Dependent Variable: Rebellions } \\
	[.1cm]\cmidrule(lr){2-5}
	;
#delimit cr
*** Table notes
run "Program/Analysis/generalnotes.do"
global note_sample "The sample consists of 536 counties in the six provinces around the canal from 1776 to 1825."
global note_year "$Year$ is the numerical year number. "
global notes "\begin{tablenotes}\footnotesize{\item \textit{Note.}$note_sample $note_dep $note_along $note_post $note_std $note_conley }\end{tablenotes}"

In [10]:
$head

\ is not a valid command name


r(199);


In [11]:
**************************************************************************
*** Export Table to Latex
**************************************************************************
esttab est1 est2 est3 est4 ///
	   using "Results/Tables/table2.tex", booktabs nonotes compress label nomtitles ///
                                   cells(b(fmt(4)) se(fmt(4) par(( ))) sesp(fmt(4) par([ ]) drop(_cons))) ///
							       collabels("",none) ///
                                   stats(depavg N N_g r2_a,fmt(4 %7.0fc 0 4) labels("Mean of the Dependent Variable" "No. of Observations" "No. of Counties" "Adjusted R-squared")) ///
								   indicate("County FE =0.OBJECTID" "Year FE=0.year" "Pre-reform rebellion $\times$ Year FE=0.year#c.ashprerebels" "Province $\times$ Year FE=0.provid#0.year"  "Prefecture Year Trend=0.prefid#c.year") ///
    							   prehead($head ) ///
                                   postfoot(\bottomrule\end{tabular}$notes\end{threeparttable}\end{table}) ///
                                   starlevels(* 0.10 ** 0.05 *** 0.01) ///
                                   replace


(file Results/Tables/table2.tex not found)
(output written to Results/Tables/table2.tex)


In [13]:
esttab est1 est2 est3 est4, ///
       cells(b(fmt(4)) se(fmt(4) par(( ))) sesp(fmt(4) par([ ]) drop(_cons))) ///
       collabels("",none) ///
       stats(depavg N N_g r2_a,fmt(4 %7.0fc 0 4) labels("Mean of the Dependent Variable" "No. of Observations" "No. of Counties" "Adjusted R-squared")) ///
       indicate("County FE =0.OBJECTID" "Year FE=0.year" "Pre-reform rebellion $\times$ Year FE=0.year#c.ashprerebels" "Province $\times$ Year FE=0.provid#0.year"  "Prefecture Year Trend=0.prefid#c.year") ///
       prehead($head ) ///
       postfoot($notes) ///
       starlevels(* 0.10 ** 0.05 *** 0.01) ///
       replace


\begin{table}[htb]\centering     \caption{Canal closure and rebellions: pre-trea
> tment trends }  \begin{threeparttable}  \begin{tabular}{l*{4}{c}}  \toprule\to
> prule  &\multicolumn{4}{c}{Dependent Variable: Rebellions } \\  [.1cm]\cmidrul
> e(lr){2-5} 
                      (1)          (2)          (3)          (4)
             ash~ypop1600 ash~ypop1600 ash~ypop1600 ash~ypop1600
----------------------------------------------------------------
pretrend           0.0003      -0.0000      -0.0002      -0.0003
                 (0.0006)     (0.0006)     (0.0006)     (0.0007)
                 [0.0007]     [0.0006]     [0.0006]     [0.0007]
_cons             -0.0523       0.0255       0.0753       0.0804
                 (0.1400)     (0.1372)     (0.1522)     (0.1689)
County FE             Yes          Yes          Yes          Yes
Year FE               Yes          Yes          Yes          Yes
Pre-reform~e           No          Yes          Yes          Yes
Province $~E           No  